In [27]:
import pandas as pd
import numpy as np

In [28]:
companies_selected = pd.read_excel("Companies.xlsx")

In [29]:
# since TSR is in % we will grade wight in % as well
grade_weight = {"A": 90, "B": 80, "C": 70, "D": 60, "E": 50, "F": 40}
companies_selected["GradeWeight"] = companies_selected["Grade"].map(grade_weight)

In [30]:
def calculate_grade_score(data, x):
    return (1 - x) * data["TSR"] + x * data["GradeWeight"]

In [31]:
# default values set as: 16% for industry allocation and 4.5% mac allocation
# beta value for potfolio should be below 1
def monte_carlo_simulation(
    data,
    max_industry_allocation=0.16,
    max_stock_allocation=0.045,
    target_beta=1.0,
    iterations=100,
):

    best_portfolio = None
    best_portfolio_beta = float("inf")

    for _ in range(iterations):
        # we first want to normalise the TSR:
        data["Industry TSR total"] = data.groupby("Industry")["TSR"].transform("sum")
        data["weighted TSR"] = data["TSR"] / data["Industry TSR total"]
        # The weightage we are giving x will range from 0.9 to 0.95
        # This will give us cases where our grade is not the main factor and TSR is which provides us with different scores on multiple iterations
        x = np.random.uniform(0.9, 0.95)

        # execute the function mention above
        data["Grade Score"] = data.apply(calculate_grade_score, axis=1, args=(x,))

        # obtain a weight scross all the investments. Basically getting a ratio so values for the weights will be between 0 and 1
        # basic normalisation
        data["Weight"] = data["Grade Score"] / data["Grade Score"].sum()

        # Lets focus on each industry
        # what is the current weight allocated to each industry
        data["Industry Allocation"] = data.groupby("Industry")["Weight"].transform(
            "sum"
        )
        data["Allocation_by_restriction"] = np.where(
            data["Industry Allocation"] > max_industry_allocation,
            max_industry_allocation,
            data["Industry Allocation"],
        )

        data["Normalised Weight"] = np.where(
            data["Allocation_by_restriction"] == 0.16,
            (data["Weight"] / data["Industry Allocation"]) * 0.16,
            data["Weight"],
        )

        # NEED TO ADD WEIGHT FOR THE MAX ALLOCATION

        # Calculate Portfolio Beta
        portfolio_beta = (data["Beta Value"] * data["Normalised Weight"]).sum()

        if portfolio_beta < target_beta:
            best_portfolio_beta = portfolio_beta
            print(x)
            break

    return best_portfolio_beta, data

In [32]:
final_beta, ideal_portfolio = monte_carlo_simulation(companies_selected)

0.9119991417347636


In [33]:
final_beta

0.8164360307328773

In [34]:
ideal_portfolio["Normalised Weight"].sum()

0.8397032027753492

In [35]:
ideal_portfolio.groupby("Industry")["Normalised Weight"].sum()

Industry
Consumer Staples    0.092976
Energy              0.099524
Finance             0.105494
Healthcare          0.106780
Industruals         0.084434
Real Estate         0.094396
Technology          0.160000
Utilities           0.096100
Name: Normalised Weight, dtype: float64

In [36]:
ideal_portfolio["final_weight"] = (
    ideal_portfolio["Normalised Weight"] / ideal_portfolio["Normalised Weight"].sum()
) * 100

In [37]:
round(ideal_portfolio["final_weight"].sum(), 2) == 100.00

True

In [38]:
max(ideal_portfolio["final_weight"])

11.561093774173473

In [39]:
ideal_portfolio.sort_values("final_weight", ascending=False).head(15)

,Company Name,Industry,Beta Value,Grade,TSR,GradeWeight,Industry TSR total,weighted TSR,Grade Score,Weight,Industry Allocation,Allocation_by_restriction,Normalised Weight,final_weight
48,APLD,Technology,1.55,B,16871.83,80,21711.92,0.777077,1557.695452,0.194338,0.320297,0.160000,0.097079,11.561094
44,NVDA,Technology,1.76,C,2694.86,70,21711.92,0.124119,300.989933,0.037551,0.320297,0.160000,0.018758,2.233924
25,Eli Lilly and Company (LLY),Healthcare,0.42,C,675.16,70,1435.04,0.470482,123.254599,0.015377,0.106780,0.106780,0.015377,1.831269
13,Raymond James Financial (RJF),Finance,1.00,A,196.37,90,1525.19,0.128751,99.360651,0.012396,0.105494,0.105494,0.012396,1.476262
20,UnitedHealth Group Incorporated (UNH),Healthcare,0.59,A,159.18,90,1435.04,0.110924,96.087899,0.011988,0.106780,0.106780,0.011988,1.427637
40,TSLA,Technology,2.12,A,1236.11,90,21711.92,0.056932,190.858664,0.023811,0.320297,0.160000,0.011895,1.416538
35,Exxon Mobil Corporation (XOM),Energy,0.88,A,119.31,90,1085.06,0.109957,92.579305,0.011550,0.099524,0.099524,0.011550,1.375507
17,Morgan Stanley (MS),Finance,1.34,B,219.69,80,1525.19,0.144041,92.292840,0.011514,0.105494,0.105494,0.011514,1.371251
15,Goldman Sachs (GS),Finance,1.35,B,211.02,80,1525.19,0.138357,91.529872,0.011419,0.105494,0.105494,0.011419,1.359915
4,Otter Tail (OTTR),Utilities,0.69,A,87.72,90,358.64,0.244591,89.799358,0.011203,0.096100,0.096100,0.011203,1.334204
